_Autor:_    __Jesús Casado__ <br> _Revisión:_ __4/12/2019__ <br>

__Introducción__<br>


__Cosas que arreglar__ <br>
* El mapa climático no cubre por completo el polígono de la cuenca.
* Rehacer mapas climáticos sobre la malla de MODIS.

***

__Índice__ <br>

In [1]:
import os
import numpy as np
import pandas as pd
import numpy.ma as ma
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from netCDF4 import Dataset
from datetime import date, datetime, timedelta
from shapely.geometry.polygon import Polygon
from matplotlib import path
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from pyproj import Proj, Transformer, CRS

In [2]:
rutaBase = os.getcwd().replace('\\', '/') + '/'

In [3]:
os.chdir(rutaBase + '../../py')
from funciones_raster import *
os.chdir(rutaBase)

## 1 Cartografía

In [4]:
# ruta cartografía
rutaGIS = os.path.join(rutaBase, '../../data/GIS/')#'C:/Users/jcr693/OneDrive - UNICAN - Estudiantes/Proyectos/IAHR2020/GIS/'

# importar polígonos de las cuencas
cuenca = gpd.read_file(rutaGIS + "mask_465_deva.shp")

# importar mdt de la cuenca
DEM = read_ascii(rutaGIS + 'dem_465_deva.asc', crs=25830)
DEM.variable = 'elevación'
DEM.units = 'm.s.n.m'

DriverError: C:/Users/jcr693/Documents/GitHub/Calibrar/Nieve/notebooks/../../data/GIS/mask_465_deva.shp: No such file or directory

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

cuenca.boundary.plot(ax=ax, edgecolor='k')
ax.imshow(DEM.data, cmap='pink', extent=DEM.extent)

ax.set_title('DEM cuenca del DEVA-1265', fontsize=13, fontweight='bold')
ax.set_aspect('equal')
ax.axis('off');

## 2 Clima: precipitación y temperatura

### 2.1 Cargar datos

In [ ]:
# Cargar netCDF con los datos climáticos
rutaData = 'C:/Users/jcr693/OneDrive - UNICAN - Estudiantes/Proyectos/IAHR2020/data/'
nc = Dataset(rutaData + 'clima_PdE.nc', 'r', format='NETCDF4')

# extraer variables
X = nc['X'][:].data
Y = nc['Y'][:].data
dates = nc['time'][:].data
dates = [datetime(1979, 1, 1).date()  + timedelta(days=int(d)) for d in dates]
pcp = nc['pcp'][::]
pcp[pcp.mask] = np.nan
pcp = pcp.data
tmp = nc['tmp'][::]
tmp[tmp.mask] = np.nan
tmp = tmp.data

nc.close()

### 2.1 Tratar datos

In [ ]:
# crear objeto PCP, reproyectar y recortar
PCP = raster3D(pcp, X, Y, dates, variable='precipitación', units='mm', label='P',
            crs=CRS.from_epsg(25830))
PCP.reproyectar(DEM.crs, DEM.cellsize, n_neighbors=4, weights='distance', p=2,
                       snap=DEM, inplace=True)
PCP.recortar(cuenca, buffer=10, inplace=True)

In [ ]:
# crear objeto TMP, reproyectar y recortar
TMP = raster3D(tmp, X, Y, dates, variable='temperatura', units='°C', label='T',
            crs=CRS.from_epsg(25830))
TMP.reproyectar(DEM.crs, DEM.cellsize, n_neighbors=4, weights='distance', p=2,
                       snap=DEM, inplace=True)
TMP.recortar(cuenca, buffer=10, inplace=True)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(10, 5))
t = -1000

for i, (Var, ax) in enumerate(zip([PCP, TMP], axes)):
    ax.imshow(Var.data[t,:,:], cmap='viridis', extent=np.array(Var.extent()))
    cuenca.boundary.plot(ax=ax, edgecolor='k')
    ax.set_title('{0} ({1}) {2}'.format(Var.variable, Var.units,
                                         Var.times[t].strftime('%d-%m-%Y')),
                 fontsize=13)
    #ax.set_aspect('equal')
    ax.axis('off');

### 2.3 Exportar datos tratados

In [ ]:
# declara el netCDF
rutaExport = os.path.join(rutaBase, '../../data/clima/')
if os.path.exists(rutaExport) is False:
    os.makedirs(rutaExport)
nc = Dataset(rutaExport + 'clima_1265.nc', 'w', format='NETCDF4')

# definir dimensiones
time = nc.createDimension('time', len(PCP.times))
x = nc.createDimension('x', len(PCP.X))
y = nc.createDimension('y', len(PCP.Y))

# definir variables
times = nc.createVariable('time', 'i2', ('time',))
times.units = 'days since {0}'.format(PCP.times[0])
#times.calendar = "gregorian"
xs = nc.createVariable('X', 'i4', ('x',))
xs.units = 'm'
ys = nc.createVariable('Y', 'i4', ('y',))
ys.units = 'm'
P = nc.createVariable('pcp', 'f4', ('time', 'y', 'x'))
P.units = 'mm'
T = nc.createVariable('tmp', 'f4', ('time', 'y', 'x'))
T.units = '°C'

# atributos
nc.description = 'Caracterización climática de la cuenca alta del río Deva'
nc.history = 'Created on {0}'.format(date.today())
np.source = 'Jesús Casado Rodríguez'

# asignar valores a las variables
xs[:] = PCP.X
ys[:] = PCP.Y
times[:] = range(len(PCP.times))
P[:,:,:] = PCP.data
T[:,:,:] = TMP.data

nc.close()